In [1]:
# Choose one of the matplotlib initialization for interactive or non-interactive
#%matplotlib qt5
import matplotlib
matplotlib.use('Qt5Agg')

import os
import numpy as np
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.geom
import matplotlib.ticker as plticker
from astropy.visualization import (ZScaleInterval, SqrtStretch, AsinhStretch, ImageNormalize)
import astropy.units as u
import matplotlib.pyplot as plt

import astropy

from astropy.table import Table
from matplotlib.backends.backend_pdf import PdfPages


In [2]:
cwpRepo = '/home/gkovacs/data/repo_DM-17825/ingested/rerun/proc_2019-02-21'
cwpTemplateRepo = '/home/gkovacs/data/repo_DM-17825/templates'
my_dbName = '/home/gkovacs/data/repo_DM-17825/ingested/rerun/proc_2019-02-21/association.db'
mrawls_dbName = '/home/gkovacs/data/repo_DM-17825/mrawls_cw_processed2/association.db'

butlerCwp = dafPersist.Butler(cwpRepo)
butlerCwpTemplate = dafPersist.Butler(cwpTemplateRepo)

In [3]:
patchList = ['10,8', '11,8', '12,8', '13,8',
             '10,7', '11,7', '12,7', '13,7',
             '10,9', '11,9', '12,9', '13,9',
             '10,5', '11,5', '12,5', '13,5',
             '10,6', '11,6', '12,6', '13,6',
             '10,10', '11,10', '12,10', '13,10']

In [4]:
def loadAllPpdbObjects(repo, dbName='association.db'):
    """Load select DIAObject columns from a PPDB into a pandas dataframe.

    Parameters
    ----------
    repo : `str`
        Path to an output repository from an ap_pipe run.
    dbName : `str`, optional
        Name of the PPDB, which must reside in (or relative to) repo.

    Returns
    -------
    objTable : `pandas.DataFrame`
        DIA Object Table containing only objects with validityEnd NULL.
        Columns selected are presently hard-wired here.
    """
    connection = sqlite3.connect(os.path.join(repo, dbName))

    # These are the tables available in the ppdb
    tables = {'obj': 'DiaObject', 'src': 'DiaSource', 'ccd': 'CcdVisit'}

    # Only get objects with validityEnd NULL because that means they are still valid
    objTable = pd.read_sql_query('select diaObjectId, ra, decl, nDiaSources, \
                                  gPSFluxMean, validityEnd, flags from {0} \
                                  where validityEnd is NULL;'.format(tables['obj']), connection)
    return objTable
# ---
def defMiniRegion(objTable):
    miniRegion = ((objTable['decl'] < -5.6) & (objTable['decl'] > -5.8) & 
               (objTable['ra'] > 155.2) & (objTable['ra'] < 155.3) &
               (objTable['nDiaSources'] > 2))
    return miniRegion
# ---
def plotMiniRegion(objTable, miniRegion, title=None):
    print('Plotting {0} DIAObjects'.format(len(objTable.loc[miniRegion, 'ra'])))
    fig = plt.figure(figsize=(7,5))
    ax1 = fig.add_subplot(111)
    cb1 = ax1.scatter((objTable.loc[miniRegion, 'ra'].values*u.deg).to_value(u.rad),
                      (objTable.loc[miniRegion, 'decl'].values*u.deg).to_value(u.rad),
                      marker='.', lw=0, s=objTable.loc[miniRegion, 'nDiaSources']*8,
                      c=objTable.loc[miniRegion, 'flags'],  #c=objTable.loc[miniRegion, 'nDiaSources'],
                      alpha=0.5, )
                      #cmap=plt.cm.get_cmap('viridis'))
    binMax = np.max(objTable['nDiaSources'].values)
    #cbplot = plt.colorbar(cb1, ax=ax1)
    #cbplot.set_label('Number of DIASources')
    #cbplot.set_clim(0, binMax)
    #cbplot.solids.set_edgecolor("face")
    plt.xlabel('RA (rad)')
    plt.ylabel('Dec (rad)')
    #plt.xlim([155.3, 155.2])
    #plt.ylim([-5.8, -5.6])
    plt.xlim([2.71040, 2.70875])
    plt.ylim([-0.1014, -0.0978])
    if title:
        plt.title(title)
# ---
def load_sources(repo, obj, sqliteFile='association.db'):
    connection = sqlite3.connect(os.path.join(repo, sqliteFile))
    tables = {'obj': 'DiaObject', 'src': 'DiaSource', 'ccd': 'CcdVisit'}
    srcTable = pd.read_sql_query('select diaSourceId, diaObjectId, ccdVisitId, midPointTai, \
                                 apFlux, psFlux, apFluxErr, psFluxErr, totFlux, totFluxErr, flags \
                                 from {1} where diaObjectId = {0};'.format(obj, tables['src']), connection)
    connection.close()
    return(srcTable)
# ---
def plot_images(repo, templateRepo, obj, patch, objTable, cutoutIdx = 0,
                    plotAllCutouts=False, diffimType='deepDiff_differenceExp',pdfWriter=None):
    sources = load_sources(repo, obj)
    ra = objTable.loc[objTable['diaObjectId'] == obj, 'ra']
    dec = objTable.loc[objTable['diaObjectId'] == obj, 'decl']
    flags = sources['flags']    
    dataIds = sources['ccdVisitId'].values  # these are ints
    dataIdDicts = []
    for dataId in dataIds:
        visit = dataId // 100
        ccdnum = dataId % 100
        dataIdDict = {'visit': visit, 'ccdnum': ccdnum}
        dataIdDicts.append(dataIdDict)
    centerSource = lsst.geom.SpherePoint(ra, dec, lsst.geom.degrees)
    size = lsst.geom.Extent2I(100, 100)
    

    print('DIAObject ID:', obj)
    #print('Flags:', flags)
    print('RA (deg):', ra.values)
    print('Dec (deg):', dec.values)
    print('DIASource IDs:', sources['diaSourceId'].values)
    src_ids = ' '.join(list(str(sources['diaSourceId'].values)))
    print('Data IDs:', dataIdDicts)

    fig=plt.figure(figsize=(10,4))
    fig.suptitle('DIAObject ID:{}'.format(obj))
    
    # light curve with psFlux by default (uses totFlux if useTotFlux=True)
#     plt.subplot(212)
#     plt.xlabel('Time (MJD)', size=16)
#     if not useTotFlux:
#         plt.errorbar(sources['midPointTai'], sources['psFlux']*1e9, yerr=sources['psFluxErr']*1e9, 
#                      ls=':', marker='o', color='#2979C1')
#         plt.ylabel('Difference Flux (nJy)', size=16)
#     else:
#         plt.errorbar(sources['midPointTai'], ources['totFlux']*1e9, yerr=sources['totFluxErr']*1e9,
#                      ls=':', marker='o', color='#2979C1')
#         plt.ylabel('Flux (nJy)', size=16)
#     plt.gca().spines['top'].set_visible(False)
#     plt.gca().spines['right'].set_visible(False)    
    
    
    # First image is for summary
    
    # processed image
    ax = fig.add_subplot(1,3,1)
    fig.subplots_adjust(left=0.05, right=0.98, bottom=0.05, wspace=0.15)
    
    butler = dafPersist.Butler(repo)
    calexpFirst = butler.get('calexp', dataIdDicts[cutoutIdx])
    ax.set_title('{visit} {ccdnum:02d}'.format(**dataIdDicts[cutoutIdx]))
    calexpCutout = calexpFirst.getCutout(centerSource, size);
    bbox = calexpCutout.getBBox()
    extentR = (bbox.getMaxY()+0.5, bbox.getMinY()-0.5, bbox.getMaxX()+0.5, bbox.getMinX()-0.5)
    calexpArray = calexpCutout.getMaskedImage().getImage().getArray()
    calexpNorm = ImageNormalize(calexpArray, interval=ZScaleInterval(), stretch=AsinhStretch())
    ax.imshow(calexpArray.T[::-1,::-1], origin='lower', cmap='gray', norm=calexpNorm, extent = extentR)
    
    # template image
    ax = fig.add_subplot(1,3,2)
    ax.set_title(patch)
    templateDataId = {'filter': 'g', 'tract': 0, 'patch': patch}
    butlerTemplate = dafPersist.Butler(templateRepo)
    template = butlerTemplate.get('deepCoadd', dataId=templateDataId)
    templateCutout = template.getCutout(centerSource, size)
    bbox = templateCutout.getBBox()
    extent = (bbox.getMinX()-0.5, bbox.getMaxX()+0.5, bbox.getMinY()-0.5, bbox.getMaxY()+0.5)
    templateArray = templateCutout.getMaskedImage().getImage().getArray()
    templateNorm = ImageNormalize(templateArray, interval=ZScaleInterval(), stretch=AsinhStretch())
    # The template seems to have the usual orientation by default
    ax.imshow(templateArray, origin='lower', cmap='gray', norm=templateNorm, extent=extent)
    #plt.imshow(np.fliplr(np.rot90(templateArray)), cmap='gray', norm=templateNorm)
    
    # difference image
    ax = fig.add_subplot(1,3,3)

    diffimFirst = butler.get(diffimType, dataIdDicts[cutoutIdx])
    diffimCutout = diffimFirst.getCutout(centerSource, size)
    bbox = diffimCutout.getBBox()
    extentR = (bbox.getMaxY()+0.5, bbox.getMinY()-0.5, bbox.getMaxX()+0.5, bbox.getMinX()-0.5)
    diffimArray = diffimCutout.getMaskedImage().getImage().getArray()
    diffimNorm = ImageNormalize(diffimArray, interval=ZScaleInterval(), stretch=AsinhStretch())
    
    ax.imshow(diffimArray.T[::-1,::-1], origin='lower', cmap='gray', norm=diffimNorm, extent = extentR)

    if pdfWriter is not None:
        pdfWriter.savefig(fig)
        plt.close(fig)
    
    n_plots_per_fig = 4
    n_figs = len(dataIdDicts) // 4
    if len(dataIdDicts) % 4 !=0:
        n_figs += 1
    fig_idx = 1
    
    if plotAllCutouts:
        fig = None
        dataIdDicts.reverse()
        n_panels = len(dataIdDicts)
        panel_idx = 1
        
        while len(dataIdDicts) > 0:
            if fig is None:
                fig = plt.figure(figsize=(10,5.5))            
                fig.subplots_adjust(left=0.05, right=0.98, bottom=0.05, wspace=0.15, hspace=0.05)
                fig.suptitle('DIAObject {}; {}/{}'.format(obj,fig_idx,n_figs))
                fig_idx += 1             
                splot_idx = 1
            
            dataIdDict = dataIdDicts.pop()
            
            # Upper row calexp
            ax = fig.add_subplot(2,n_plots_per_fig,splot_idx)
            calexpFirst = butler.get('calexp', dataIdDict)
            ax.set_title('{visit} {ccdnum:02d}; {idx}/{n_panels}'.format(idx=panel_idx, n_panels=n_panels,**dataIdDict))
            calexpCutout = calexpFirst.getCutout(centerSource, size);
            bbox = calexpCutout.getBBox()
            extentR = (bbox.getMaxY()+0.5, bbox.getMinY()-0.5, bbox.getMaxX()+0.5, bbox.getMinX()-0.5)
            calexpArray = calexpCutout.getMaskedImage().getImage().getArray()
            calexpNorm = ImageNormalize(calexpArray, interval=ZScaleInterval(), stretch=AsinhStretch())
            ax.imshow(calexpArray.T[::-1,::-1], origin='lower', cmap='gray', norm=calexpNorm, extent = extentR)

            # Bottom row diffim
            ax = fig.add_subplot(2,n_plots_per_fig,splot_idx + n_plots_per_fig)
                         
            diffimFirst = butler.get(diffimType, dataIdDict)
            diffimCutout = diffimFirst.getCutout(centerSource, size)
            bbox = diffimCutout.getBBox()
            extentR = (bbox.getMaxY()+0.5, bbox.getMinY()-0.5, bbox.getMaxX()+0.5, bbox.getMinX()-0.5)
            diffimArray = diffimCutout.getMaskedImage().getImage().getArray()
            diffimNorm = ImageNormalize(diffimArray, interval=ZScaleInterval(), stretch=AsinhStretch())
    
            ax.imshow(diffimArray.T[::-1,::-1], origin='lower', cmap='gray', norm=diffimNorm, extent = extentR)

            panel_idx += 1
            splot_idx += 1
            if splot_idx > n_plots_per_fig:
                if pdfWriter is not None:
                    pdfWriter.savefig(fig)
                    plt.close(fig)
                fig = None
                 
        if fig is not None and pdfWriter is not None:
            pdfWriter.savefig(fig)
            plt.close(fig)
            
#             diffim = butler.get(diffimType, dataId)
#             diffimArray = diffim.getCutout(centerSource, size).getMaskedImage().getImage().getArray()
#             diffimNorm = ImageNormalize(diffimArray, interval=ZScaleInterval(), stretch=SqrtStretch())
#             plt.subplot(10, 10, idx+1)
#             plt.gca().get_xaxis().set_ticks([])
#             plt.gca().get_yaxis().set_ticks([])
#             plt.imshow(np.rot90(np.fliplr(calexpArray)), cmap='gray', norm=calexpNorm)
#             if labelCutouts:
#                 if idx == 0:
#                     plt.text(1, 26, 'Proc', color='lime', size=8)
#                 plt.text(2, 5, str(sources['midPointTai'][idx])[1:8], color='lime', size=8)            
#             plt.subplot(10, 10, idx+50+1)
#             plt.gca().get_xaxis().set_ticks([])
#             plt.gca().get_yaxis().set_ticks([])
#             plt.imshow(np.rot90(np.fliplr(diffimArray)), cmap='gray', norm=diffimNorm)
#             if labelCutouts:
#                 if idx == 0:
#                     plt.text(1, 26, 'Diff', color='lime', size=8)
#                 plt.text(2, 5, str(sources['midPointTai'][idx])[1:8], color='lime', size=8)
    

def patchFinder(obj, objTable, templateButler, patchList):
    for patch in patchList:
        ra = objTable.loc[objTable['diaObjectId'] == obj, 'ra']
        dec = objTable.loc[objTable['diaObjectId'] == obj, 'decl']
        centerSource = lsst.geom.SpherePoint(ra, dec, lsst.geom.degrees)
        size = lsst.geom.Extent2I(30, 30)
        templateDataId = {'filter': 'g', 'tract': 0, 'patch': patch}
        templateImage = templateButler.get('deepCoadd', dataId=templateDataId)
        try:
            cutout = templateImage.getCutout(centerSource, size)
        except:
            continue
        else:
            templatePatch = patch
            #print('template patch:', templatePatch)
            #print('object id:', obj)
            return templatePatch
            break

In [5]:
cwpObjTable = loadAllPpdbObjects(cwpRepo)
cwpMiniRegion = defMiniRegion(cwpObjTable)
cwpMiniUnflagged = cwpMiniRegion & (cwpObjTable['flags'] == 0)
cwpObjList = list(cwpObjTable.loc[cwpMiniRegion, 'diaObjectId'])
cwpObjList.sort()

In [6]:
t_cwpMiniUnflagged=Table.from_pandas(cwpObjTable.loc[cwpMiniUnflagged])

# Find the patch that belongs to the mini region
patch = patchFinder(t_cwpMiniUnflagged['diaObjectId'][0],cwpObjTable,butlerCwpTemplate,patchList)
print(patch)

11,8


Looping through the objects and plotting them in a pdf file

In [ ]:
plot_images(cwpRepo,cwpTemplateRepo,cwpObjList[1],patch,cwpObjTable,plotAllCutouts=True)

In [ ]:
plt.show()

In [7]:
with PdfPages('proc_2019-02-21_diffims.pdf') as W:
    for obj in cwpObjList:
        print(obj)
        plot_images(cwpRepo,cwpTemplateRepo,obj,patch,cwpObjTable,plotAllCutouts=True,pdfWriter=W)

176492674482700417
DIAObject ID: 176492674482700417
RA (deg): [155.28781788]
Dec (deg): [-5.63162922]
DIASource IDs: [176492674482700417 176516726299558044 176538201136038025
 176638703370764492 176660178207244460 176703557376934077
 176811361056063609 176832835892543633 176854740225753223
 176876215062233243 176897689898713297 177175144786034963
 177196619622514920 177218094458994870 177239569295474891
 177660905587212508 177680232940044406 178382030596211233
 178403505432691027 180303598964442945 181077552071180427
 176493765404393973 176517817221251336 176539292057731271
 176553894946537727 176639794292457751 176661269128937937
 176682743965418286 176704648298628037 176726123135107346
 176834356310966537 176855831147446473 176877305983926531
 176898780820406580 176985539159785698 177176235707728326
 177197710544208418 177240660217168550 177661996508905690
 177681323861737684 178404596354384241 180304689886134518]
Data IDs: [{'visit': 410929, 'ccdnum': 3}, {'visit': 410985, 'ccdnum':

176492687367602397
DIAObject ID: 176492687367602397
RA (deg): [155.23181196]
Dec (deg): [-5.7077967]
DIASource IDs: [176492687367602397 176682743965418195 176704648298627924]
Data IDs: [{'visit': 410929, 'ccdnum': 6}, {'visit': 411371, 'ccdnum': 57}, {'visit': 411422, 'ccdnum': 57}]
176492687367602398
DIAObject ID: 176492687367602398
RA (deg): [155.23223134]
Dec (deg): [-5.71395397]
DIASource IDs: [176492687367602398 176538214020939993 176638716255666401
 176681665928626392 176725045098316014 176811373940965601
 176854753110655165 176876227947135185 177175157670936733
 177196632507416718 177660918472114340 177680245824946372
 180303611849344138 176493765404393923 176661269128937897
 176682743965418194 176704648298627922 176812451977756847
 176834356310966524 176877305983926510]
Data IDs: [{'visit': 410929, 'ccdnum': 6}, {'visit': 411035, 'ccdnum': 6}, {'visit': 411269, 'ccdnum': 6}, {'visit': 411369, 'ccdnum': 6}, {'visit': 411470, 'ccdnum': 6}, {'visit': 411671, 'ccdnum': 6}, {'visit'

176492687367602407
DIAObject ID: 176492687367602407
RA (deg): [155.27000425]
Dec (deg): [-5.79691858]
DIASource IDs: [176492687367602407 176516739184459949 176538214020940005
 176552816909746349 176638716255666412 176660191092146425
 176681665928626401 176703570261835987 176725045098316025
 176811373940965611 176832848777445622 176854753110655175
 176876227947135195 176897702783615180 177066065501618302
 177175157670936740 177196632507416725 177218107343896736
 177239582180376751 177660918472114349 177680245824946381
 178382043481112810 178403518317592725 180303611849344232
 181077564956082384 176493778289295586 176517830106153182
 176539304942633212 176553907831439588 176639807177359598
 176661282013839623 176682756850319664 176704661183529265
 176726136020009228 176812464862658724 176834369195868385
 176855844032348383 176877318868828413 176898793705308396
 176985552044687470 177067156423311442 177176248592629947
 177197723429109914 177219198265589918 177262147938549879
 177662009393

176493765404393884
DIAObject ID: 176493765404393884
RA (deg): [155.20075574]
Dec (deg): [-5.66253023]
DIASource IDs: [176493765404393884 176661269128937860 176682743965418128
 176704648298627854]
Data IDs: [{'visit': 410931, 'ccdnum': 57}, {'visit': 411321, 'ccdnum': 57}, {'visit': 411371, 'ccdnum': 57}, {'visit': 411422, 'ccdnum': 57}]
176493765404393885
DIAObject ID: 176493765404393885
RA (deg): [155.20138315]
Dec (deg): [-5.66408916]
DIASource IDs: [176493765404393885 176661269128937861 176682743965418129
 176704648298627856]
Data IDs: [{'visit': 410931, 'ccdnum': 57}, {'visit': 411321, 'ccdnum': 57}, {'visit': 411371, 'ccdnum': 57}, {'visit': 411422, 'ccdnum': 57}]
176493765404393887
DIAObject ID: 176493765404393887
RA (deg): [155.20297244]
Dec (deg): [-5.64303096]
DIASource IDs: [176493765404393887 176517817221251309 176539292057731248
 176661269128937862 176812451977756839 176898780820406550
 176985539159785663 177176235707728245 177197710544208317
 177219185380688059 17726213505

176493765404393908
DIAObject ID: 176493765404393908
RA (deg): [155.22163472]
Dec (deg): [-5.61857882]
DIASource IDs: [176493765404393908 176661269128937888 176682743965418168]
Data IDs: [{'visit': 410931, 'ccdnum': 57}, {'visit': 411321, 'ccdnum': 57}, {'visit': 411371, 'ccdnum': 57}]
176493765404393909
DIAObject ID: 176493765404393909
RA (deg): [155.2216398]
Dec (deg): [-5.62355439]
DIASource IDs: [176493765404393909 176517817221251317 176553894946537707
 176639794292457728 176661269128937886 176682743965418166
 176726123135107323 176834356310966522 176898780820406559
 177197710544208346 177240660217168447 177262135053648278
 181078642992873785]
Data IDs: [{'visit': 410931, 'ccdnum': 57}, {'visit': 410987, 'ccdnum': 57}, {'visit': 411071, 'ccdnum': 57}, {'visit': 411271, 'ccdnum': 57}, {'visit': 411321, 'ccdnum': 57}, {'visit': 411371, 'ccdnum': 57}, {'visit': 411472, 'ccdnum': 57}, {'visit': 411724, 'ccdnum': 57}, {'visit': 411874, 'ccdnum': 57}, {'visit': 412570, 'ccdnum': 57}, {'vi

176493765404393931
DIAObject ID: 176493765404393931
RA (deg): [155.24046481]
Dec (deg): [-5.61959418]
DIASource IDs: [176493765404393931 176661269128937903 176682743965418211]
Data IDs: [{'visit': 410931, 'ccdnum': 57}, {'visit': 411321, 'ccdnum': 57}, {'visit': 411371, 'ccdnum': 57}]
176493765404393935
DIAObject ID: 176493765404393935
RA (deg): [155.24234608]
Dec (deg): [-5.68119373]
DIASource IDs: [176493765404393935 176517817221251322 176553894946537711
 176639794292457733 176661269128937904 176682743965418214
 176704648298627945 176726123135107328 176812451977756850
 176834356310966527 176877305983926513 176898780820406564
 176985539159785689 177197710544208372 177240660217168491
 177661996508905677 178404596354384219 181078642992873793]
Data IDs: [{'visit': 410931, 'ccdnum': 57}, {'visit': 410987, 'ccdnum': 57}, {'visit': 411071, 'ccdnum': 57}, {'visit': 411271, 'ccdnum': 57}, {'visit': 411321, 'ccdnum': 57}, {'visit': 411371, 'ccdnum': 57}, {'visit': 411422, 'ccdnum': 57}, {'visi

176493765404393953
DIAObject ID: 176493765404393953
RA (deg): [155.25882695]
Dec (deg): [-5.69585676]
DIASource IDs: [176493765404393953 176517817221251327 176539292057731262
 176553894946537717 176639794292457738 176661269128937919
 176682743965418244 176704648298627982 176726123135107334
 176812451977756853 176834356310966529 176855831147446467
 176877305983926520 176985539159785692 177067143538409698
 177092054348726491 177176235707728309 177197710544208394
 177219185380688072 177262135053648348 177661996508905680
 177681323861737676 178383121517904242 178404596354384228
 180304689886134507 181078642992873801]
Data IDs: [{'visit': 410931, 'ccdnum': 57}, {'visit': 410987, 'ccdnum': 57}, {'visit': 411037, 'ccdnum': 57}, {'visit': 411071, 'ccdnum': 57}, {'visit': 411271, 'ccdnum': 57}, {'visit': 411321, 'ccdnum': 57}, {'visit': 411371, 'ccdnum': 57}, {'visit': 411422, 'ccdnum': 57}, {'visit': 411472, 'ccdnum': 57}, {'visit': 411673, 'ccdnum': 57}, {'visit': 411724, 'ccdnum': 57}, {'vis

176516726299558043
DIAObject ID: 176516726299558043
RA (deg): [155.28531855]
Dec (deg): [-5.68495379]
DIASource IDs: [176516726299558043 176638703370764491 176660178207244458
 176681653043724486 176703557376934076 176854740225753222
 176876215062233241 177218094458994867 177680232940044405]
Data IDs: [{'visit': 410985, 'ccdnum': 3}, {'visit': 411269, 'ccdnum': 3}, {'visit': 411319, 'ccdnum': 3}, {'visit': 411369, 'ccdnum': 3}, {'visit': 411420, 'ccdnum': 3}, {'visit': 411772, 'ccdnum': 3}, {'visit': 411822, 'ccdnum': 3}, {'visit': 412618, 'ccdnum': 3}, {'visit': 413694, 'ccdnum': 3}]
176516739184459938
DIAObject ID: 176516739184459938
RA (deg): [155.21685366]
Dec (deg): [-5.72583707]
DIASource IDs: [176516739184459938 176538214020939989 176638716255666396
 176660191092146412 176681665928626387 176703570261835976
 176725045098316009 176811373940965597 176832848777445608
 176876227947135181 176897702783615169 177175157670936730
 177239582180376742 177660918472114336 177680245824946368
 1

KeyboardInterrupt: 

In [ ]:
pwd

================

In [ ]:
my_conn = sqlite3.connect(my_dbName)
mr_conn = sqlite3.connect(mrawls_dbName)

In [ ]:
plotMiniRegion(cwpObjTable, cwpMiniRegion, title='CompareWarp')